In [1]:
from backtester import Backtester
import pandas as pd
import numpy as np

### Input parameters for Backtester: asset data frame, strategy function, starting balance (optional)

In [2]:
# Read a sample asset data as data frame
df = pd.read_csv('data_2020_2023_3min.csv')
# Shorten the data frame to 10k rows
df = df.iloc[:100000]
display('Asset Data Frame', df, df.dtypes)

# Create a sample strategy function that takes asset df as input and outputs list of non-overlapping trade signals
def alternating_trading_strategy(df, trade_interval=10):
    """
    A simple trading strategy that alternates between long and short trades at a fixed interval, ensuring each trade is closed.

    Parameters:
        df (pd.DataFrame): The dataframe containing market data.
        trade_interval (int): The number of periods between trades.

    Returns:
        list: A list of trading signals, where `1` represents a buy signal, `-1` represents a sell signal,
              and `0` represents no action.
    """
    signals = [0] * len(df)  # Initialize all signals to 0
    in_trade = False  # Indicates if currently in a trade
    trade_type = 1  # Start with a long trade

    for i in range(0, len(df)):
        if i % trade_interval == 0:
            signals[i] = trade_type
            in_trade = not in_trade  # Toggle the in_trade status
            if in_trade:
                # If we just entered a trade, prepare to close it in the next interval
                trade_type = -trade_type
        else:
            signals[i] = 0

    # Close any open trade at the end
    if in_trade:
        signals[-1] = -trade_type

    return signals

# Apply the strategy on the asset df and generate list of signals
signals = alternating_trading_strategy(df)
# Display unique signals, positions and their frequencies
signal_series = pd.Series(signals)
position_series = signal_series.cumsum()
signals_df = pd.DataFrame({'Signals': signal_series.value_counts()})
positions_df = pd.DataFrame({'Positions': position_series.value_counts()})
display('Signals', signals_df, 'Positions', positions_df)

'Asset Data Frame'

,datetime,open,high,low,close,volume
0,2020-01-01 05:30:00,7195.24,7196.25,7180.26,7182.43,70.572637
1,2020-01-01 05:33:00,7183.83,7188.94,7178.20,7179.99,37.399739
2,2020-01-01 05:36:00,7180.00,7187.74,7179.99,7187.68,30.886999
3,2020-01-01 05:39:00,7187.68,7193.53,7186.02,7188.71,38.880878
4,2020-01-01 05:42:00,7189.52,7189.52,7180.24,7180.97,25.202615
...,...,...,...,...,...,...
9995,2020-01-22 01:15:00,8558.37,8565.80,8533.00,8550.93,255.486450
9996,2020-01-22 01:18:00,8550.93,8555.13,8520.50,8541.24,286.542518
9997,2020-01-22 01:21:00,8541.20,8543.97,8505.63,8521.95,311.942336
9998,2020-01-22 01:24:00,8523.36,8552.27,8520.63,8550.77,235.787323


datetime     object
open        float64
high        float64
low         float64
close       float64
volume      float64
dtype: object

'Signals'

,Signals
0,9000
1,500
-1,500


'Positions'

,Positions
0,5000
1,2500
-1,2500


### See the backtester in action

In [3]:
bt = Backtester(asset_df=df, strategy=alternating_trading_strategy)

# Show trade book
display('Trade Book', bt.tb_df)

# Generate trade ledgers with different modes
bt.generate_ledger(mode='static')
bt.generate_ledger(mode='compounding')

# Show trade ledgers
display('Static Ledger', bt.sl_df)
display('Compounding Ledger', bt.cl_df)

# Summarise backtest results, view long and short trades separately
bt.generate_summary_df(positional_components=True)
display('Backtest Summary', bt.summary_df)

'Trade Book'

,start_time,end_time,position,entry,crest,trough,end,win,change_p,spike_p,dip_p,duration
0,2020-01-01 05:30:00,2020-01-01 06:00:00,1,7182.43,7193.53,7175.47,7183.69,1,0.017543,0.154544,-0.096903,0 days 00:30:00
1,2020-01-01 06:30:00,2020-01-01 07:00:00,-1,7185.45,7180.18,7230.00,7218.65,0,-0.462045,0.073343,-0.620003,0 days 00:30:00
2,2020-01-01 07:30:00,2020-01-01 08:00:00,1,7216.91,7238.88,7211.41,7232.77,1,0.219762,0.304424,-0.076210,0 days 00:30:00
3,2020-01-01 08:30:00,2020-01-01 09:00:00,-1,7230.25,7220.10,7241.40,7241.39,0,-0.154075,0.140382,-0.154213,0 days 00:30:00
4,2020-01-01 09:30:00,2020-01-01 10:00:00,1,7226.76,7229.88,7215.03,7226.31,0,-0.006227,0.043173,-0.162313,0 days 00:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...
495,2020-01-21 20:30:00,2020-01-21 21:00:00,-1,8656.43,8652.76,8671.25,8659.81,0,-0.039046,0.042396,-0.171202,0 days 00:30:00
496,2020-01-21 21:30:00,2020-01-21 22:00:00,1,8643.29,8663.47,8630.00,8631.64,0,-0.134787,0.233476,-0.153761,0 days 00:30:00
497,2020-01-21 22:30:00,2020-01-21 23:00:00,-1,8631.46,8626.48,8652.45,8636.27,0,-0.055726,0.057696,-0.243180,0 days 00:30:00
498,2020-01-21 23:30:00,2020-01-22 00:00:00,1,8647.00,8648.91,8625.86,8628.36,0,-0.215566,0.022089,-0.244478,0 days 00:30:00


'Static Ledger'

,position,change_p,deployed,balance,pnl
0,1,0.017543,10000,10001.754281,1.754281
1,-1,-0.462045,10000,9955.549798,-46.204483
2,1,0.219762,10000,9977.525963,21.976164
3,-1,-0.154075,10000,9962.118473,-15.407489
4,1,-0.006227,10000,9961.495788,-0.622686
...,...,...,...,...,...
495,-1,-0.039046,10000,9819.074605,-3.904612
496,1,-0.134787,10000,9805.595941,-13.478664
497,-1,-0.055726,10000,9800.023303,-5.572638
498,1,-0.215566,10000,9778.466694,-21.556609


'Compounding Ledger'

,position,change_p,deployed,balance,pnl
0,1,0.017543,10000.000000,10001.754281,1.754281
1,-1,-0.462045,10001.754281,9955.541693,-46.212588
2,1,0.219762,9955.541693,9977.420155,21.878462
3,-1,-0.154075,9977.420155,9962.047455,-15.372699
4,1,-0.006227,9962.047455,9961.427133,-0.620322
...,...,...,...,...,...
495,-1,-0.039046,9788.680819,9784.858719,-3.822100
496,1,-0.134787,9784.858719,9771.670037,-13.188682
497,-1,-0.055726,9771.670037,9766.224640,-5.445398
498,1,-0.215566,9766.224640,9745.171971,-21.052669


'Backtest Summary'

,n_trades,accuracy,aagr,cagr,static_drawdown,comp_drawdown,max_spike_p,max_dip_p,max_win_p,avg_win_p,min_win_p,max_loss_p,avg_loss_p,min_loss_p,avg_rr,worst_rr,max_duration,average_duration,min_duration
long_trades,250,52.0,0.687933,417.377633,10.024423,10.466267,2.471815,-3.008056,2.092711,0.301925,0.002581,-1.100430,-0.248608,-0.006227,0.823410,3.644709,0 days 00:30:00,0 days 00:30:00,0 days 00:30:00
short_trades,250,48.0,-0.838213,-89.202181,10.082659,10.509277,1.514566,-1.577642,1.360311,0.214399,0.000719,-1.374899,-0.286172,-0.007875,1.334765,6.412811,0 days 00:30:00,0 days 00:30:00,0 days 00:30:00
all_trades,500,50.0,-0.075140,-31.587597,10.199044,10.634417,2.471815,-3.008056,2.092711,0.259913,0.000719,-1.374899,-0.268141,-0.006227,1.031660,5.289850,0 days 00:30:00,0 days 00:30:00,0 days 00:30:00
